In [ ]:
import random
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from datasets import load_dataset

seed = 2025
random.seed(seed)

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/junyuye/Library/Python/3.9/lib/python/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [28]:
system_prompt = """你是一个情感陪伴助手，善于体会用户情感需求。你能够和用户对话的同时感受用户的情感，
并遵循给定的【聊天历史】做出符合正常行为逻辑，顺应用户情绪的回答。回答中包含的情绪需要与【聊天历史】中的情绪变化一致。
若没有【聊天历史】则自己感受用户话语中的情感，进行符合用户情感的体贴答复。"""

dataset = load_dataset("gauishou233/friend_dialog_dataset")
print(len(dataset["train"]), len(dataset["test"]))

train_dataset = dataset["train"].shuffle().select(range(len(dataset["train"])//10))
test_dataset = dataset["test"].shuffle().select(range(1000))
del dataset

6820506 10000


In [ ]:
from typing import List, Optional
from pprint import pprint
from tqdm import tqdm

model_name = "Qwen/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)  # Qwen 需要支持远程代码
# tokenizer.pad_token = tokenizer.eos_token
print("BOS token:", tokenizer.bos_token)
print("EOS token:", tokenizer.eos_token)


def truncate_if_odd(lst):
    if len(lst) % 2 != 0:
        return lst[:-1]
    return lst

def to_chatml(
    turns: List[str],
    system_prompt: Optional[str] = None,
    is_train_data: bool = False
) -> str:
    chatml = []

    # if system prompt
    if system_prompt is not None:
        chatml.append("<|im_start|>system")
        chatml.append(system_prompt)
        chatml.append("<|im_end|>")

    # even idx -> user，odd idx -> assistant
    turns = truncate_if_odd(turns)
    for idx, text in enumerate(turns):
        role = "user" if idx % 2 == 0 else "assistant"
        chatml.append(f"<|im_start|>{role}")
        chatml.append(text)
        chatml.append("<|im_end|>")

    # eos
    if is_train_data:
        chatml.append("<|endoftext|>")

    return "\n".join(chatml) + "\n"


train_examples = [to_chatml(ex['dialog']) for ex in tqdm(train_dataset)]

BOS token: None
EOS token: <|im_end|>


100%|██████████| 682050/682050 [00:25<00:00, 27243.21it/s]


In [37]:
import json
with open('./sentiment.jsonl', 'w') as f:
    for t in train_examples:
        text = {"content": t}
        f.write(json.dumps(text, ensure_ascii=False) + '\n')
